In [ ]:
import os
import site
os.sys.path.insert(0, '/home/schirrmr/braindecode/code/')
os.sys.path.insert(0, '/home/schirrmr/braindecode/code/braindecode/')
os.sys.path.insert(0, '/home/schirrmr/code/auto-diagnosis/')
%cd /home/schirrmr/
# switch to cpu
os.environ['THEANO_FLAGS'] = 'floatX=float32,device=cpu,nvcc.fastmath=True'

%load_ext autoreload
%autoreload 2
import numpy as np
import matplotlib
from matplotlib import pyplot as plt
from matplotlib import cm
%matplotlib inline
%config InlineBackend.figure_format = 'png' 
matplotlib.rcParams['font.size'] = 7

import matplotlib.lines as mlines
import seaborn
seaborn.set_style('darkgrid')
matplotlib.rcParams['figure.figsize'] = (12.0, 3.0)
import logging
import importlib
importlib.reload(logging) # see https://stackoverflow.com/a/21475297/1469195
log = logging.getLogger()
log.setLevel('DEBUG')
import sys
logging.basicConfig(format='%(asctime)s %(levelname)s : %(message)s',
                     level=logging.DEBUG, stream=sys.stdout)
from numpy.random import RandomState
from braindecode.torch_ext.util import np_to_var
from autodiag.perturbation import combine_covs, combine_vars

In [ ]:
import os.path
n_recordings = None#20 # only if I want to limit it
    
folder = 'data/models/pytorch/auto-diag/less-minutes/12/'
from hyperoptim.rerun import rerun_exp
ex = rerun_exp(folder, update_params=dict(only_return_exp=True), 
          save_experiment=False)

In [ ]:
exp = ex.result
n_stride = exp.iterator.n_preds_per_input#5400
n_stride

In [ ]:
assert hasattr(exp, 'test_dataset')
if n_recordings is not None:
    exp.test_dataset.n_recordings = n_recordings
test_X, test_y = exp.test_dataset.load()
from braindecode.datautil.signal_target import SignalAndTarget
test_set = SignalAndTarget(test_X, test_y)


In [ ]:
labels = test_set.y#data/models/pytorch/auto-diag/less-minutes/12/test_trial_labels.npy


In [ ]:
cat data/models/pytorch/auto-diag/less-minutes/12/config.json

In [ ]:
import numpy as np
preds_per_trial = np.load(os.path.join(folder,'test_crop_preds.npy'))

In [ ]:
preds_per_trial = [np.concatenate(p, axis=1) for p in preds_per_trial]

In [ ]:
n_receptive_field = exp.iterator.input_time_length -n_stride + 1

In [ ]:
n_receptive_field

In [ ]:

all_trial_accs = []
all_crop_accs = []
minute_steps = (0.2,1,2,4,8,16)
for minutes in minute_steps:
    n_stop = int(round(minutes * 60 * 100 - n_receptive_field + 1))
    print("n_stop", n_stop)
    reduced_preds = [p[:,:n_stop] for p in preds_per_trial]
    lengths = np.array([p.shape[1] for p in reduced_preds])
    assert np.max(lengths) == n_stop
    mean_preds_per_trial = [np.mean(preds, axis=(1)) for preds in
                                reduced_preds]
    accuracy = np.mean(np.argmax(mean_preds_per_trial, axis=1) == labels)
    all_trial_accs.append(accuracy)
    print("Accuracy for {:.0f}: {:.1f}".format(minutes, accuracy * 100))
    
    crop_accs = [np.mean(i == np.argmax(p, axis=0)) for i,p in zip(labels, reduced_preds)]
    crop_acc = np.sum(crop_accs * lengths / np.sum(lengths))
    print("CropAcc. for {:.0f}: {:.1f}".format(minutes, crop_acc * 100))
    all_crop_accs.append(crop_acc)

In [ ]:
filename = os.path.join(folder, 'reduced_time_accuracies.npy')
log.info("Save to {:s}".format(filename))

np.save(filename, np.array([minute_steps, all_trial_accs, all_crop_accs]))

In [ ]:

crop_preds = np.load(os.path.join(folder,'test_crop_preds.npy'))
trial_preds = [np.concatenate(c[:-1], axis=1) for c in crop_preds]
max_shape = np.max([list(t.shape) for t in trial_preds], axis=0)
preds_padded = np.full((len(trial_preds), max_shape[1], max_shape[0]),np.nan)
for i_trial, trial_pred in enumerate(trial_preds):
    preds_padded[i_trial,:trial_pred.shape[1]] =  trial_pred.T

nan_mask = (0 * preds_padded + 1)[:,:,0]

labels_padded = np.argmax(preds_padded, axis=2)

pred_correct = labels_padded == labels[:,None]

pred_correct = pred_correct * nan_mask

correct_timecourse = np.nanmean(pred_correct, axis=0) * 100
correct_std_timecourse = np.nanstd(pred_correct, axis=0) * 100

In [ ]:
plt.plot((600 + np.arange(len(correct_timecourse))) / 100.0,correct_timecourse)
plt.fill_between((600 + np.arange(len(correct_timecourse))) / 100.0,
                 correct_timecourse - correct_std_timecourse / np.sqrt(len(pred_correct)),
                 correct_timecourse + correct_std_timecourse / np.sqrt(len(pred_correct)),
            alpha=0.3)
plt.xlabel('Time [sec]', fontsize=14)
plt.ylabel('Accuracy [%]', fontsize=14)